In [9]:
from ReadLoad import read_jsonl, write_jsonl, read_json, write_json
from llama_index.core.node_parser import SentenceWindowNodeParser
from llama_index.core import Document
from tqdm import tqdm

test_dataB = read_jsonl("dataset/test-B-0722.json")

In [10]:
documents = [Document(text=t['条款'],
                      metadata={
                          '产品名': t['产品名'],
                          'ID': t['ID'],
                          '问题': t['问题']
                      }) for t in test_dataB]
for data in tqdm(documents):
    data.text = data.text.replace('。','. ')
    data.text = data.text.replace('！','! ')
    data.text = data.text.replace('？','? ')
    
# documents[:5]

100%|██████████| 1000/1000 [00:00<00:00, 14238.69it/s]


In [11]:
from llama_index.core.node_parser import SentenceWindowNodeParser
from llama_index.core import Document

parser = SentenceWindowNodeParser.from_defaults(
    # how many sentences on either side to capture
    window_size=3,
    # the metadata key that holds the window of surrounding sentences
    window_metadata_key="window",
    # the metadata key that holds the original sentence
    original_text_metadata_key="original_sentence"
)
nodes = parser.get_nodes_from_documents(documents[:5])


In [12]:
import os
from dotenv import dotenv_values
from llama_index.legacy.llms import OpenAILike as OpenAI
from llama_index.core import Settings
config = dotenv_values(".env")

    # 初始化 LLM 嵌入模型 和 Reranker
llm = OpenAI(
    model_name="qwen2-7b-instruct",
    api_key=config['qwen_key'],
    api_base=config['qwen_url'],
    is_chat_model=True
)
Settings.llm = llm

In [13]:
from modelscope import snapshot_download
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

model_dir = snapshot_download('Xorbits/bge-small-zh-v1.5')
embeding = HuggingFaceEmbedding(
        model_name=model_dir,
        cache_folder="./",
        embed_batch_size=512,
    )
Settings.embed_model = embeding

2024-07-24 18:36:19,974 - modelscope - WARNING - Model revision not specified, use revision: v0.0.2
Downloading: 100%|██████████| 349/349 [00:00<00:00, 670B/s]
Downloading: 100%|██████████| 91.4M/91.4M [00:02<00:00, 47.1MB/s]
Downloading: 100%|██████████| 317/317 [00:00<00:00, 464B/s]
Downloading: 100%|██████████| 52.0/52.0 [00:00<00:00, 91.8B/s]
Downloading: 100%|██████████| 125/125 [00:00<00:00, 204B/s]
Downloading: 100%|██████████| 429k/429k [00:00<00:00, 770kB/s]
Downloading: 100%|██████████| 367/367 [00:00<00:00, 631B/s]
Downloading: 100%|██████████| 107k/107k [00:00<00:00, 196kB/s]


In [14]:
from llama_index.core import VectorStoreIndex
index = VectorStoreIndex(nodes)

In [15]:
from llama_index.core.response.notebook_utils import display_source_node
from llama_index.core.vector_stores import (
    MetadataFilter,
    MetadataFilters,
    FilterOperator,
)
query = test_dataB[1]['问题']
filters = MetadataFilters(
    filters=[
        MetadataFilter(
            key="问题", value=query
        ),
    ]
)
vector_retriever = index.as_retriever(similarity_top_k=2,filters=filters)
retrieve_nodes = vector_retriever.retrieve(query)
print(query)
for node in retrieve_nodes:
    print(node.id_)
    print(node.text)
    print(node.metadata['window'])
    print(node.score)
    print("=====")

保险公司在何种情况下不承担意外伤害导致的保险责任？
97159639-d549-4337-bca5-3daa3993d185
保险金申请人未能提供有关材料，导致保险人无法核实该申请的真实性的，保险人对无法核实部分不承担给付保险金的责任. 

保险金额一经确定，保险期间内不能进行变更. 
 保险金的申请
保险金申请人（释义九）向保险人申请给付保险金时，应提供如下材料：
（一）保险金给付申请书；
（二）保险合同凭据；
（三）保险金申请人的有效身份证件；
（四）支持索赔的全部账单、证明、信息和证据，医院出具的病历资料、医学诊断书、处方、病理检查、化验检查报告、医疗费用原始单据、费用明细单据等.  保险金申请人因特殊原因不能提供上述材料的，应提供其他合法有效的材料；
（五）保险金申请人所能提供的其他与确认保险事故的性质、原因等有关的其他证明和资料；
（六）若保险金申请人委托他人申请的，还应提供授权委托书原件、委托人和受托人的身份证明等相关证明文件. 
 保险金申请人未能提供有关材料，导致保险人无法核实该申请的真实性的，保险人对无法核实部分不承担给付保险金的责任. 
 意外伤害
指以外来的、突发的、非本意的和非疾病的客观事件为直接且单独的原因致使身体受到的伤害.  自然死亡、疾病身故、猝死、自杀以及自伤均不属于意外伤害. 
 医院
指经中华人民共和国卫生部门审核认定的二级及二级以上的公立医院或保险人扩展承保的医疗机构，除另有约定外，仅限于上述医院的普通部，不包括如下机构或医疗服务：
（一）特需医疗、外宾医疗、干部病房、联合病房、国际医疗中心、VIP部、联合医院、A级病房；
（二）诊所、康复中心、家庭病床、护理机构；
（三）休养、戒酒、戒毒中心；
（四）保险人不予理赔的医疗机构. 

0.783183763982253
=====
9f7a9125-bc32-4bd4-ab06-3439150525ec
保险金申请人未能提供有关材料，导致保险人无法核实该申请的真实性的，保险人对无法核实部分不承担给付保险金的责任. 

（三）投保人申报的被保险人年龄不真实，导致投保人支付保险费多于应付保险费的，保险人应将多收的保险费无息退还投保人. 
 保险金的申请
保险金申请人（释义十五）向保险人申请给付保险金时，应提供如下材料.  保险金申请人因特殊原因不能提供以下材料的，应提供其他合法有效

In [16]:
from llama_index.core import PromptTemplate

new_text_qa_template_str = (
    """\
    上下文信息如下：
    ----------
    {context_str}
    ----------
    根据上下文信息而非先验知识，构建一个经过严谨思考且内容详实的答案，来回答问题。
    充分运用上下文信息来支撑你的答案，并确保回答符合人类的偏好以及遵循指示的原则。
    如果上下文信息没有相关知识，可以回答不确定，不要复述上下文信息。
    
    问题：{query_str}
    回答：\
    """
)

In [21]:
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.vector_stores import ExactMatchFilter, MetadataFilters,MetadataFilter

def get_query_engine(query):

    filters = MetadataFilters(
        filters=[
            MetadataFilter(
                key="问题", value=query
            ),
        ]
    )
    vector_retriever = index.as_retriever(similarity_top_k=2,filters=filters)

    text_qa_template = PromptTemplate(new_text_qa_template_str)
    query_engine = RetrieverQueryEngine.from_args(
            retriever=vector_retriever,
            #llm=llm,
            text_qa_template=text_qa_template,
            #refine_template=refine_template,
            response_mode="compact"
            )
    return query_engine

In [22]:
query = test_dataB[1]['问题']
query_engine = get_query_engine(query)
response = query_engine.query(query)

ValueError: Cannot use llm_chat_callback on an instance without a callback_manager attribute.